## Project I: Analyze the behaviors of the StackOverFlow Users

## REQUIRED: Loading the data from MongoDB source and Normalization beforeAll

In [1]:
!pip install findspark

In [2]:
import findspark
findspark.init()

In [3]:
%%time
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Declare the spark context
spark = SparkSession \
    .builder \
    .master('local[4]') \
    .appName("ASM1") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1') \
    .getOrCreate()

# Load data to df
def read_data(spark, source, schema, table_name):
    # Read the data from corresponding collections in Mongo
    df = spark.read \
        .format(source) \
        .option("uri", f"mongodb://127.0.0.1:27017/ASM1.{table_name}")\
        .load()
    
    # Handle the "NA" value in `OwnerUserId`
    df = df.withColumn("OwnerUserId", 
                       when(df["OwnerUserId"] == "NA", None).otherwise(df["OwnerUserId"])
                      )
    
    # Cast the columes in each collection to the desired data type
    if table_name == "Answer":
        df = df.select(col("Id").cast("int"), 
                       col("OwnerUserId").cast("int"), 
                       col("CreationDate").cast("date"),
                       col("ParentId").cast("int"),
                       col("Score").cast("int"),
                       col("Body").cast("string")
                      )
    else:
        df = df.select(col("Id").cast("int"), 
                       col("OwnerUserId").cast("int"), 
                       col("CreationDate").cast("date"),
                       col("ClosedDate").cast("date"),
                       col("Score").cast("int"),
                       col("Title").cast("string"),
                       col("Body").cast("string")
                      )
        
    return spark.createDataFrame(df.rdd, schema)

# Declare the Struct type for answer df
schema_ans = StructType([
    StructField("Id", IntegerType()),
    StructField("OwnerUserId", IntegerType()),
    StructField("CreationDate", DateType()),
    StructField("ParentId", IntegerType()),
    StructField("Score", IntegerType()),
    StructField("Body", StringType())
])

# Load the data to ans_df
ans_df = read_data(spark, 'com.mongodb.spark.sql.DefaultSource', schema_ans, "Answer")
ans_df.printSchema()
ans_df.show(100)

# Declare the Struct type for question df
schema_quest = StructType([
    StructField("Id", IntegerType()),
    StructField("OwnerUserId", IntegerType()),
    StructField("CreationDate", DateType()),
    StructField("ClosedDate", DateType()),
    StructField("Score", IntegerType()),
    StructField("Title", StringType()),
    StructField("Body", StringType())
])

# Load the data to quest_df
quest_df = read_data(spark, 'com.mongodb.spark.sql.DefaultSource', schema_quest, "Question")
quest_df.printSchema()
quest_df.show(100)


root
 |-- Id: integer (nullable = true)
 |-- OwnerUserId: integer (nullable = true)
 |-- CreationDate: date (nullable = true)
 |-- ParentId: integer (nullable = true)
 |-- Score: integer (nullable = true)
 |-- Body: string (nullable = true)
+----+-----------+------------+--------+-----+--------------------+
|  Id|OwnerUserId|CreationDate|ParentId|Score|                Body|
+----+-----------+------------+--------+-----+--------------------+
|  92|         61|  2008-08-01|      90|   13|<p><a href="http:...|
| 124|         26|  2008-08-01|      80|   12|<p>I wound up usi...|
| 199|         50|  2008-08-01|     180|    1|<p>I've read some...|
| 269|         91|  2008-08-01|     260|    4|<p>Yes, I thought...|
| 307|         49|  2008-08-02|     260|   28|<p><a href="http:...|
| 332|         59|  2008-08-02|     330|   19|<p>I would be a b...|
| 344|        100|  2008-08-02|     260|    6|<p>You might be a...|
| 359|        119|  2008-08-02|     260|    5|<P>You could use ...|
| 473|     

## Requirement 1: Calculate the number of occurrences of programming languages

### Option 1: Using Native SQL

In [3]:
%%time
# Register the quest_df as a temporary view in Spark SQL
quest_df.createOrReplaceTempView("questions")

# Define a list of programming languages
programming_languages = ['Java', 'Python', 'C\+\+', 'C#', 'Go', 'Ruby', 'Javascript', 'PHP', 'HTML', 'CSS', 'SQL']

# Use a for loop to count the occurrences of each programming language and union the results
result = spark.sql(f"SELECT '{programming_languages[0]}' as Programming_Language, COUNT(*) as Count FROM questions WHERE Body LIKE '%{programming_languages[0]}%'")
for language in programming_languages[1:]:
    result = result.union(spark.sql(f"SELECT '{language}' as Programming_Language, COUNT(*) as Count FROM questions WHERE Body LIKE '%{language}%'"))

# Show the result as a data frame
result.show()


+--------------------+-----+
|Programming_Language|Count|
+--------------------+-----+
|                Java|65142|
|              Python|21578|
|                 C++|18918|
|                  C#|25037|
|                  Go|42706|
|                Ruby| 7337|
|          Javascript|11740|
|                 PHP|36538|
|                HTML|55131|
|                 CSS|22656|
|                 SQL|63072|
+--------------------+-----+

Wall time: 2min 46s


### Option 2: Using Object Expression

In [4]:
%%time
programming_languages = ['Java', 'Python', 'C++', 'C#', 'Go', 'Ruby', 'Javascript', 'PHP', 'HTML', 'CSS', 'SQL']

# Start with a dataframe containing the first programming language
result = quest_df.filter(expr(f"Body like '%{programming_languages[0]}%'")) \
                 .agg(count(lit(1)).alias("Count")) \
                 .withColumn("Programming_Language", lit(programming_languages[0]))

# Use a for loop to count the occurrences of each programming language and union the results
for language in programming_languages[1:]:
    df = quest_df.filter(expr(f"Body like '%{language}%'")) \
                 .agg(count(lit(1)).alias("Count")) \
                 .withColumn("Programming_Language", lit(language))
    result = result.union(df)

# Show the result as a data frame
result.show()

+-----+--------------------+
|Count|Programming_Language|
+-----+--------------------+
|65142|                Java|
|21578|              Python|
|18918|                 C++|
|25037|                  C#|
|42706|                  Go|
| 7337|                Ruby|
|11740|          Javascript|
|36538|                 PHP|
|55131|                HTML|
|22656|                 CSS|
|63072|                 SQL|
+-----+--------------------+

Wall time: 3min 3s


## Requirement 2: Find the most used domains in the questions

In [5]:
%%time
# Extract urls from the Body of the Questions
url_reg = r'([a-zA-Z]+\.)+([a-zA-Z]+)(\.[a-zA-Z]+)+'
url_df = quest_df.select(regexp_extract(col('Body'), url_reg, 0).alias('Domain'))

# Remove the all empty values
url_df = url_df.withColumn("Domain", lower("Domain"))\
                .withColumn("Domain", trim("Domain"))\
                .filter(col('Domain').isNotNull())\
                .filter(col('Domain') != "")

# Count the occurrences of each domain and show the first 20 most used values
url_df.createOrReplaceTempView("domains")
url_result = spark.sql("SELECT Domain, COUNT(*) as Count FROM domains GROUP BY Domain")
url_result = url_result.select("Domain", "Count").orderBy(col("Count").desc())
url_result.show(20)

+--------------------+-----+
|              Domain|Count|
+--------------------+-----+
|   i.stack.imgur.com|53322|
|  system.out.println|12449|
|  msdn.microsoft.com| 4705|
| schemas.android.com| 3520|
|    en.wikipedia.org| 3187|
| ajax.googleapis.com| 2840|
|   r.layout.activity| 2414|
|      www.google.com| 2225|
|     code.google.com| 2174|
|     www.example.com| 2111|
|developer.android...| 2022|
|developers.google...| 1830|
|         i.imgur.com| 1703|
|     www.youtube.com| 1631|
|system.collection...| 1568|
| developer.apple.com| 1336|
|schemas.microsoft...| 1273|
|     code.jquery.com| 1220|
|     gist.github.com| 1162|
|    system.out.print| 1152|
+--------------------+-----+
only showing top 20 rows

Wall time: 49.5 s


## Requirement 3: Calculate the total score of the User by day

### Option 1: Using DF API

In [6]:
%%time
#Remove the null value from ans_df and quest_df
ans_df = ans_df.filter(col("OwnerUserId").isNotNull())
quest_df = quest_df.filter(col("OwnerUserId").isNotNull())

# Define the window specification
windowSpec = Window.partitionBy(col("OwnerUserId")) \
                    .orderBy(col("OwnerUserId"), col("CreationDate")) \
                    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

# Define the DataFrame to use
df = ans_df.select(col("OwnerUserId"), col("CreationDate"), col("Score")) \
           .union(quest_df.select(col("OwnerUserId"), col("CreationDate"), col("Score")))

# Sum the Score by `OwnerUserId` and `CreationDate`
df = df.groupBy(col("OwnerUserId"), col("CreationDate")) \
        .agg(sum(col("Score")).alias("Score"))

# Calculate the total score by day using Windowing
result = df.withColumn("Score", sum("Score").over(windowSpec))
result.orderBy("OwnerUserId", "CreationDate").show()

+-----------+------------+-----+
|OwnerUserId|CreationDate|Score|
+-----------+------------+-----+
|          1|  2008-08-04|    7|
|          1|  2008-08-14|    9|
|          1|  2008-08-17|   11|
|          1|  2008-08-28|   22|
|          1|  2008-11-26|   32|
|          1|  2008-12-22|   43|
|          1|  2008-12-30|   46|
|          1|  2009-01-08|   66|
|          1|  2009-06-07|  121|
|          1|  2009-07-19|  125|
|          1|  2009-10-08|  175|
|          1|  2009-11-21|  207|
|          1|  2009-12-03|  211|
|          1|  2010-04-23|  212|
|          1|  2012-02-24|  290|
|          3|  2008-11-16|    0|
|          3|  2009-01-09|    4|
|          3|  2009-01-12|   17|
|          3|  2009-02-09|   66|
|          3|  2013-01-23|   68|
+-----------+------------+-----+
only showing top 20 rows

Wall time: 51.4 s


### Option 2: Using Spark SQL

In [9]:
%%time
#Remove the null value from ans_df and quest_df
ans_df = ans_df.filter(col("OwnerUserId").isNotNull())
quest_df = quest_df.filter(col("OwnerUserId").isNotNull())

# Register the data frames as temporary tables
ans_df.createOrReplaceTempView("ans_table")
quest_df.createOrReplaceTempView("quest_table")

# Define the SQL query to calculate the total score by day using Windowing
result = spark.sql("""
    SELECT OwnerUserId, CreationDate, SUM(Score) AS Score, SUM(SUM(Score)) OVER 
    (PARTITION BY OwnerUserId ORDER BY OwnerUserId, CreationDate ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS Total_Score 
    FROM (
        SELECT OwnerUserId, CreationDate, Score FROM ans_table 
        UNION ALL 
        SELECT OwnerUserId, CreationDate, Score FROM quest_table
    )
    GROUP BY OwnerUserId, CreationDate
    ORDER BY OwnerUserId, CreationDate
""")

result.show()

+-----------+------------+-----+-----------+
|OwnerUserId|CreationDate|Score|Total_Score|
+-----------+------------+-----+-----------+
|          1|  2008-08-04|    7|          7|
|          1|  2008-08-14|    2|          9|
|          1|  2008-08-17|    2|         11|
|          1|  2008-08-28|   11|         22|
|          1|  2008-11-26|   10|         32|
|          1|  2008-12-22|   11|         43|
|          1|  2008-12-30|    3|         46|
|          1|  2009-01-08|   20|         66|
|          1|  2009-06-07|   55|        121|
|          1|  2009-07-19|    4|        125|
|          1|  2009-10-08|   50|        175|
|          1|  2009-11-21|   32|        207|
|          1|  2009-12-03|    4|        211|
|          1|  2010-04-23|    1|        212|
|          1|  2012-02-24|   78|        290|
|          3|  2008-11-16|    0|          0|
|          3|  2009-01-09|    4|          4|
|          3|  2009-01-12|   13|         17|
|          3|  2009-02-09|   49|         66|
|         

## Requirement 4: Calculate the total number of scores gained by the User in a period of time

### Option 1: Using Dataframe API

In [14]:
%%time
from datetime import datetime as date

#Remove the null value from ans_df and quest_df
ans_df = ans_df.filter(col("OwnerUserId").isNotNull())
quest_df = quest_df.filter(col("OwnerUserId").isNotNull())

# Define the DataFrame to use by union the Question DF and the Answer DF
df = ans_df.select(col("OwnerUserId"), col("CreationDate"), col("Score")) \
           .union(quest_df.select(col("OwnerUserId"), col("CreationDate"), col("Score")))

# Define start_date and end_date as date objects
start_date = date(2008, 4, 4)
end_date = date(2010, 10, 10)

# Filter the CreationDate by the Start date and End date, sum the score by OwnerUserId
result = df.filter((col("CreationDate") >= start_date) & (col("CreationDate") <= end_date)) \
            .groupBy("OwnerUserId") \
            .agg(sum("Score").alias("TotalScore"))\
            .orderBy("OwnerUserId")\
            .show()

+-----------+----------+
|OwnerUserId|TotalScore|
+-----------+----------+
|          1|       212|
|          3|        66|
|          4|       105|
|          5|       197|
|          9|        17|
|         13|       965|
|         17|       189|
|         19|         4|
|         20|        23|
|         22|        15|
|         23|        32|
|         24|         1|
|         25|        81|
|         26|       140|
|         27|         9|
|         29|      1871|
|         30|         2|
|         33|       274|
|         34|        29|
|         35|        60|
+-----------+----------+
only showing top 20 rows

Wall time: 34.9 s


### Option 2: Using Spark SQL

In [15]:
%%time
from pyspark.sql.functions import sum

#Remove the null value from ans_df and quest_df
ans_df = ans_df.filter(col("OwnerUserId").isNotNull())
quest_df = quest_df.filter(col("OwnerUserId").isNotNull())

# Create a temporary view for the answer and question dataframes
ans_df.createOrReplaceTempView("ans")
quest_df.createOrReplaceTempView("quest")

# Union the answer and question dataframes
union_df = spark.sql("SELECT OwnerUserId, CreationDate, Score FROM ans UNION SELECT OwnerUserId, CreationDate, Score FROM quest")
union_df.createOrReplaceTempView("union_df")

# Filter by start date and end date, group by OwnerUserId and sum the score
start_date = "2008-04-04"
end_date = "2010-10-10"
result = spark.sql(f"SELECT OwnerUserId, SUM(Score) AS TotalScore FROM union_df WHERE CreationDate BETWEEN '{start_date}' AND '{end_date}' GROUP BY OwnerUserId ORDER BY OwnerUserId")

result.show()

+-----------+----------+
|OwnerUserId|TotalScore|
+-----------+----------+
|          1|       212|
|          3|        66|
|          4|       105|
|          5|       197|
|          9|        17|
|         13|       965|
|         17|       185|
|         19|         4|
|         20|        23|
|         22|        15|
|         23|        32|
|         24|         1|
|         25|        81|
|         26|       129|
|         27|         9|
|         29|      1871|
|         30|         2|
|         33|       274|
|         34|        29|
|         35|        60|
+-----------+----------+
only showing top 20 rows

Wall time: 34.6 s


## Requirement 5: Find the question has more that 5 answers

### Option 1: Using Dataframe API

In [18]:
%%time
# Set up bucketing for question and answer collections
ans_df.write.bucketBy(8, "ParentId").mode("overwrite").saveAsTable("ans_bucketed")
quest_df.write.bucketBy(8, "Id").mode("overwrite").saveAsTable("quest_bucketed")

# Perform bucketing join and count
ans_bucketed = spark.read.table("ans_bucketed").drop("Id")
quest_bucketd = spark.read.table("quest_bucketed")

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
join_expr = ans_bucketed.ParentId == quest_bucketd.Id
join_df = quest_bucketd.join(ans_bucketed, join_expr, "inner").groupBy("Id", "Title").count().filter("count > 5")

join_df.orderBy("Id") \
        .select(col("Id").alias("QuestionId"),
                col("Title").alias("QuestionTitle"),
                col("count").alias("NumberOfAnswer")
                ).show()

+----------+--------------------+--------------+
|QuestionId|       QuestionTitle|NumberOfAnswer|
+----------+--------------------+--------------+
|       180|Function for crea...|             9|
|       260|Adding scripting ...|             9|
|       330|Should I use nest...|             9|
|       580|Deploying SQL Ser...|            14|
|       650|Automatically upd...|             6|
|       930|How do I connect ...|             7|
|      1160|Use SVN Revision ...|            12|
|      1300|Is nAnt still sup...|             6|
|      1610|Can I logically r...|             8|
|      1760|.NET Unit Testing...|            11|
|      1970|What language do ...|             7|
|      2120|Convert HashBytes...|             7|
|      2300|How do I traverse...|             6|
|      2530|How do you disabl...|            38|
|      2550|What are effectiv...|             6|
|      2630|What are your fav...|            13|
|      2750|Data verification...|             8|
|      2840|Paging S

### Option 2: Using Spark SQL

In [22]:
%%time
# Set up bucketing for question and answer collections
spark.sql("DROP TABLE IF EXISTS ans_bucketed")
spark.sql("DROP TABLE IF EXISTS quest_bucketed")

ans_df.write.bucketBy(8, "ParentId").mode("overwrite").saveAsTable("ans_bucketed")
quest_df.write.bucketBy(8, "Id").mode("overwrite").saveAsTable("quest_bucketed")

# # Read table and create temp views
# ans_bucketed = spark.read.table("ans_bucketed").drop("Id")
# quest_bucketd = spark.read.table("quest_bucketed")
# ans_bucketed.createOrReplaceTempView("ans_bucketed")
# quest_bucketd.createOrReplaceTempView("quest_bucketed")

# Perform bucketing join and count
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

join_query = """
SELECT quest_bucketed.Id AS QuestionId, quest_bucketed.Title AS QuestionTitle, COUNT(*) AS NumberOfAnswer
FROM quest_bucketed 
JOIN ans_bucketed ON ans_bucketed.ParentId = quest_bucketed.Id
GROUP BY quest_bucketed.Id, quest_bucketed.Title
HAVING COUNT(*) > 5
ORDER BY quest_bucketed.Id
"""

result = spark.sql(join_query)
result.show()


+----------+--------------------+--------------+
|QuestionId|       QuestionTitle|NumberOfAnswer|
+----------+--------------------+--------------+
|       180|Function for crea...|             9|
|       260|Adding scripting ...|             9|
|       330|Should I use nest...|             9|
|       580|Deploying SQL Ser...|            14|
|       650|Automatically upd...|             6|
|       930|How do I connect ...|             7|
|      1160|Use SVN Revision ...|            12|
|      1300|Is nAnt still sup...|             6|
|      1610|Can I logically r...|             8|
|      1760|.NET Unit Testing...|            11|
|      1970|What language do ...|             7|
|      2120|Convert HashBytes...|             7|
|      2300|How do I traverse...|             6|
|      2530|How do you disabl...|            38|
|      2550|What are effectiv...|             6|
|      2630|What are your fav...|            13|
|      2750|Data verification...|             8|
|      2840|Paging S

## Requirement 6: Find the active users

### Option 1: Using .join() two times

In [23]:
%%time
#Remove the null value from ans_df and quest_df
ans_df = ans_df.filter(col("OwnerUserId").isNotNull())
quest_df = quest_df.filter(col("OwnerUserId").isNotNull())

#Filter users has more than 50 Answers or TotalScore > 500
ans_active = ans_df.groupBy("OwnerUserId")\
                    .agg(count("Id").alias("NumberOfAnswers"), sum("Score").alias("TotalScore"))\
                    .filter((col("NumberOfAnswers") > 50) | (col("TotalScore") > 500))
# ans_active.agg(count("*")).show() --> 4000

# Set up bucketing for question and answer collections
# ans_df.write.bucketBy(8, "ParentId").mode("overwrite").saveAsTable("ans_bucketed")
# quest_df.write.bucketBy(8, "Id").mode("overwrite").saveAsTable("quest_bucketed")

# Perform bucketing join and filter questions having 5 answers at the same day
ans_bucketed = spark.read.table("ans_bucketed").drop("Id")
quest_bucketd = spark.read.table("quest_bucketed")

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
join_expr = ans_bucketed.ParentId == quest_bucketd.Id
quest_active = quest_bucketd.join(ans_bucketed, join_expr, "inner")\
                            .filter(quest_bucketd["CreationDate"] == ans_bucketed["CreationDate"])\
                            .groupBy(quest_bucketd["Id"], quest_bucketd["OwnerUserId"]).count().filter("count > 5")

quest_active = quest_active.select("OwnerUserId").distinct()
# quest_active.agg(count("*")).show() --> 8000

# Join two active df to find active users
spark.conf.set("spark.sql.shuffle.partitions", 4)
active_users = ans_active.join(quest_active, ans_active.OwnerUserId == quest_active.OwnerUserId, "inner")\
                .select(ans_active["OwnerUserId"], ans_active["NumberOfAnswers"], ans_active["TotalScore"])\
                .orderBy(ans_active["OwnerUserId"])

active_users.agg(count("*")).show()
active_users.show()

+--------+
|count(1)|
+--------+
|     313|
+--------+

+-----------+---------------+----------+
|OwnerUserId|NumberOfAnswers|TotalScore|
+-----------+---------------+----------+
|         91|             68|       543|
|        184|             11|       636|
|        357|             12|       672|
|        377|             59|       204|
|        572|             56|       339|
|        987|             62|       241|
|       1175|             61|       230|
|       1288|            179|      1177|
|       1583|            739|      2789|
|       1585|             49|       826|
|       1737|             59|       201|
|       1786|             63|       651|
|       1862|             58|       178|
|       1942|             88|       421|
|       1950|             26|      1582|
|       1965|             68|       691|
|       2147|             63|       263|
|       2197|             19|       513|
|       2238|             17|       655|
|       2385|             85|       112|
+

### Option 2: Using .isin()

In [24]:
%%time
#Remove the null value from ans_df and quest_df
ans_df = ans_df.filter(col("OwnerUserId").isNotNull())
quest_df = quest_df.filter(col("OwnerUserId").isNotNull())

# Set up bucketing for question and answer collections
# ans_df.write.bucketBy(8, "ParentId").mode("overwrite").saveAsTable("ans_bucketed")
# quest_df.write.bucketBy(8, "Id").mode("overwrite").saveAsTable("quest_bucketed")

# Perform bucketing join and filter questions having 5 answers at the same day
ans_bucketed = spark.read.table("ans_bucketed")
quest_bucketd = spark.read.table("quest_bucketed")

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
join_expr = ans_bucketed.ParentId == quest_bucketd.Id
quest_active = quest_bucketd.join(ans_bucketed, join_expr, "inner")\
                            .filter(quest_bucketd["CreationDate"] == ans_bucketed["CreationDate"])\
                            .groupBy(quest_bucketd["Id"], quest_bucketd["OwnerUserId"]).count().filter("count > 5")

#Filter users has more than 50 Answers or TotalScore > 500
values_to_match = [row[0] for row in quest_active.select('OwnerUserId').distinct().collect()]
active_users = ans_bucketed.filter(ans_bucketed["OwnerUserId"].isin(values_to_match))\
                     .groupBy(ans_bucketed["OwnerUserId"])\
                     .agg(count(ans_bucketed["Id"]).alias("NumberOfAnswers"), sum(ans_bucketed["Score"]).alias("TotalScore"))\
                     .filter((col("NumberOfAnswers") > 50) | (col("TotalScore") > 500))\
                     .orderBy("OwnerUserId")

active_users.agg(count("*")).show()
active_users.show()

+--------+
|count(1)|
+--------+
|     313|
+--------+

+-----------+---------------+----------+
|OwnerUserId|NumberOfAnswers|TotalScore|
+-----------+---------------+----------+
|         91|             68|       543|
|        184|             11|       636|
|        357|             12|       672|
|        377|             59|       204|
|        572|             56|       339|
|        987|             62|       241|
|       1175|             61|       230|
|       1288|            179|      1177|
|       1583|            739|      2789|
|       1585|             49|       826|
|       1737|             59|       201|
|       1786|             63|       651|
|       1862|             58|       178|
|       1942|             88|       421|
|       1950|             26|      1582|
|       1965|             68|       691|
|       2147|             63|       263|
|       2197|             19|       513|
|       2238|             17|       655|
|       2385|             85|       112|
+